# 01. Collect Data

## Setting

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [4]:
# url = "https://pcmap.place.naver.com/restaurant/13166754/review/visitor" # 리뷰 많은 곳

url = "https://pcmap.place.naver.com/restaurant/1804344332/review/visitor" # 리뷰 적은 곳

## 1. 순차적 개발

### 페이지 접근

In [5]:
# 크롬창 열기
options = Options()
driver = webdriver.Chrome(options = options)

# 창모드 전체화면으로 크기 늘리기
driver.maximize_window()

# 로딩시간: 페이지 로딩
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

# 네이버 리뷰 주소 접속하기
driver.get(url)

# 로딩시간: 페이지 로딩
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'body')))

[<selenium.webdriver.remote.webelement.WebElement (session="ff761eee9aab8e6635680e0f79e522e7", element="f.0E9D800305A5BFBC0AC5172F3AC7FB25.d.9057217FE6F493C2523A71326240A1EE.e.299")>]

In [12]:
driver.get("https://pcmap.place.naver.com/restaurant/13166754/review/visitor")

### ➕ 최신순으로 정렬

* 추천순은 좋은 반응으로 편향될 가능성이 있다고 판단하여 최신순으로 정렬하는 과정을 추가함

In [13]:
driver.find_elements(By.CLASS_NAME, "zMIkw")[-1].click()

### 댓글 더보기

* "더 보기" 버튼을 눌러야 다음 댓글 생성됨
* 리뷰의 끝이 보이면 "더 보기" 버튼은 없어짐 ❌ 에러 발생

In [14]:
def find_element_by_class(driver, class_name):
    """element를 찾지 못하면 None으로 반환하는 함수"""
    try:
        element = driver.find_element(By.CLASS_NAME, class_name)
    except:
        element = None
    
    return element 

In [36]:
# 반복 수를 결정하는 경우
rep = 5 

for _ in range(rep):
    more_btn = find_element_by_class(driver, "fvwqf")
    
    # 더 보기 버튼 안보이면 break
    if not more_btn: break

    more_btn.click()
    time.sleep(0.5)

In [ ]:
# 목표 수집 댓글 수를 결정하는 경우
n = 50

while True:
    more_btn = find_element_by_class(driver, "fvwqf")

    # 더 보기 버튼 안보이면 break
    if not more_btn: break
    
    more_btn.click()
    time.sleep(0.5)

    # 목표 리뷰 개수 도달하면 break
    reviews = driver.find_elements(By.CLASS_NAME, "owAeM")
    print(len(reviews))
    if len(reviews) > n:
        # print("목표 리뷰 개수 도달")
        break


### 리뷰 추출하기
* 긴 글의 경우 내용 더보기 버튼이 활성화됨

In [24]:
reviews = driver.find_elements(By.CLASS_NAME, "owAeM")

In [16]:
json_data = []

for review in reviews:
    review_json = {}

    # 긴 글일 경우 내용 더보기 클릭
    text_area = review.find_elements(By.CLASS_NAME, "xHaT3")
    
    if len(text_area) == 2:
        text_area[0].click()
        time.sleep(1)

    # 텍스트가 없으면 수집하지 않기
    content = review.find_element(By.CLASS_NAME, "zPfVt")
    if not content.text:
        continue

    info_content = ""
    if find_element_by_class(review, "MnhVd"):
        additional_infos = review.find_elements(By.CLASS_NAME, "xalSr")
        info_content = ", ".join([x.text for x in additional_infos])
    
    review_json["reviewer"] = review.find_element(By.CLASS_NAME, "P9EZi").text
    review_json["review"] = content.text
    review_json["additional_info"] = info_content

    visit_keys = ["date", "n_visit", "auth_method"]
    visit_info = review.find_elements(By.CLASS_NAME, "CKUdu")
    for key, info in zip(visit_keys, visit_info):
        review_json[key] = info.text.split("\n")[-1].strip()

    json_data.append(review_json)

In [17]:
json_data[:2]

[{'reviewer': 'shooooooo',
  'review': '종류도 알차게 많고 다 맛있어요!',
  'additional_info': '예약 후 이용, 대기 시간 바로 입장, 데이트, 연인·배우자',
  'date': '2024년 6월 7일 금요일',
  'n_visit': '1번째 방문',
  'auth_method': '영수증'},
 {'reviewer': 'gudrmfl2',
  'review': '맛나요',
  'additional_info': '예약 후 이용, 대기 시간 10분 이내, 데이트, 연인·배우자',
  'date': '2024년 6월 7일 금요일',
  'n_visit': '1번째 방문',
  'auth_method': '영수증'}]

### 데이터 프레임 만들기

In [18]:
review_df = pd.json_normalize(json_data)
review_df.head()

,reviewer,review,additional_info,date,n_visit,auth_method
0,shooooooo,종류도 알차게 많고 다 맛있어요!,"예약 후 이용, 대기 시간 바로 입장, 데이트, 연인·배우자",2024년 6월 7일 금요일,1번째 방문,영수증
1,gudrmfl2,맛나요,"예약 후 이용, 대기 시간 10분 이내, 데이트, 연인·배우자",2024년 6월 7일 금요일,1번째 방문,영수증
2,코코리383,맛있어요,"예약 없이 이용, 대기 시간 바로 입장, 데이트, 연인·배우자",2024년 5월 28일 화요일,1번째 방문,영수증
3,vivajin,좋아요,,2024년 5월 25일 토요일,1번째 방문,영수증
4,illiiilillil,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다ㅎㅎ ...,"예약 후 이용, 대기 시간 바로 입장, 데이트, 연인·배우자",2024년 5월 19일 일요일,1번째 방문,영수증


### 저장

In [119]:
# json data 저장
import json 

with open("./data/naver_review_raw_data.json", "w") as json_file:
    json.dump(json_data, json_file, indent=4, ensure_ascii=False)

In [115]:
# 데이터 프레임 저장
review_df.to_csv("./data/naver_review_raw_data.csv")

## 2. 최종 코드 정리

### 함수 모음

In [16]:
import logging
import time
import json
import pandas as pd
from pathlib import Path, WindowsPath
from typing import List, Literal, Optional
from review_analyzer.selenium_tool import *

from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Logger 설정
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# 터미널에 출력할 수 있게 함
handler = logging.StreamHandler()
formatter = logging.Formatter('%(levelname)s: %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

class ReviewCrawler:
    """네이버 리뷰를 크롤링하는 클래스

    Attributes:
        url (str): 크롤링할 네이버 리뷰 페이지 URL
        n_target (Optional[int]): 목표로 하는 리뷰 수집 개수. 기본값은 200.
        sort_type (Literal["latest", "recommended"]): 리뷰 정렬 방식(최신순, 추천순)
        no_data (bool): 리뷰 데이터를 추가 수집할 수 있는 상태 유무. 기본값은 False
        driver (Optional[WebDriver]): Selenium WebDriver 인스턴스. _open_chrome 메서드에 의해 생성됨
        json_data (List[Dict[str,str]]): json 형식의 리뷰 데이터. _crawling 메서드에 의해 생성됨 
        data (pd.DataFrame): DataFrame 형식의 리뷰 데이터. _crawling 메서드에 의해 생성됨
    
    Methods:
        run(): 리뷰 크롤링 실행
        save_data(save_path): 리뷰 데이터를 save_path에 저장

    Private Methods:
        _open_chrome(): Chrome 브라우저를 생성
        _sort_and_scroll(): 리뷰 정렬 및 더보기 클릭
        _crawling(reviews): 리뷰 데이터 추출 및 수집
        _add_crawling(n_click): 리뷰 데이터 추가 수집
    """

    def __init__(
        self, 
        url: str, 
        n_target: Optional[int] = 200, 
        sort_type: Literal["latest", "recommended"] = "latest"
    ):
        self.url = url 
        self.n_target = n_target
        self.sort_type = sort_type 
        self.no_data = False
        self.driver = None 
        self.json_data = []
        self.data = None
    

    def _open_chrome(self) -> WebDriver:
        """Chrome 브라우저를 생성하는 메서드

        Returns:
            WebDriver : Selenium WebDriver 인스턴스
        """
        # 크롬창 열기
        options = Options()
        driver = webdriver.Chrome(options = options)

        # 창모드 전체화면으로 크기 늘리기
        driver.maximize_window()

        # 로딩시간: 페이지 로딩
        loading(driver)

        # 네이버 리뷰 주소 접속하기
        driver.get(self.url)

        # 로딩시간: 페이지 로딩
        loading(driver)

        return driver

    def _sort_and_scroll(self) -> List[WebElement]:
        """리뷰 정렬 방식과 더보기를 클릭하는 메서드

        Returns:
            List[WebElement] : 리뷰에 해당하는 WebElement 객체 리스트
        """
        # 최신순으로 정렬
        if self.sort_type == "latest":
            self.driver.find_elements(By.CLASS_NAME, "zMIkw")[-1].click()

        # 목표 수집 댓글 수를 결정하는 경우
        while True:
            more_btn = find_element_by_class(self.driver, "fvwqf")

            # 더 보기 버튼 안보이면 break
            if not more_btn: 
                self.no_data = True
                break
            
            more_btn.click()
            time.sleep(0.5)

            # 목표 리뷰 개수 도달하면 break
            reviews = self.driver.find_elements(By.CLASS_NAME, "owAeM")
            if len(reviews) > self.n_target:
                # print("목표 리뷰 개수 도달")
                break
        
        # 전체 리뷰 추출
        reviews = self.driver.find_elements(By.CLASS_NAME, "owAeM")
        
        return reviews

    def _crawling(self, reviews: List[WebElement]) -> None:
        """리뷰 데이터 추출 및 수집하는 메서드

        Args:
            reviews (List[WebElement]): 리뷰에 해당하는 WebElement 객체 리스트
        """
        for review in reviews:
            review_json = {}

            text_area = review.find_elements(By.CLASS_NAME, "xHaT3")
            # 긴 글일 경우 내용 더보기 클릭
            if len(text_area) == 2:
                text_area[0].click()
                time.sleep(1)

            content = review.find_element(By.CLASS_NAME, "zPfVt")
            # 텍스트가 없으면 수집하지 않기
            if not content.text:
                continue
            
            # 세부 정보 추출
            info_content = ""
            if find_element_by_class(review, "MnhVd"):
                additional_infos = review.find_elements(By.CLASS_NAME, "xalSr")
                info_content = ", ".join([x.text for x in additional_infos])
            
            # 리뷰 데이터 저장
            review_json["reviewer"] = review.find_element(By.CLASS_NAME, "P9EZi").text
            review_json["review"] = content.text
            review_json["additional_info"] = info_content

            visit_keys = ["date", "n_visit", "auth_method"]
            visit_info = review.find_elements(By.CLASS_NAME, "CKUdu")
            for key, info in zip(visit_keys, visit_info):
                review_json[key] = info.text.split("\n")[-1].strip()

            # 데이터에 추가
            self.json_data.append(review_json)

    def _add_crawling(self, n_click: int) -> None:
        """리뷰 데이터를 추가 수집하는 메서드

        Args:
            n_click (int): 더보기 버튼 클릭 수
        """
        for _ in range(n_click):
            more_btn = find_element_by_class(self.driver, "fvwqf")

            # 더 보기 버튼 안보이면 break
            if not more_btn: break
            
            more_btn.click()
            time.sleep(0.5)
        
        reviews = self.driver.find_elements(By.CLASS_NAME, "owAeM")[-n_click*10:]
        self._crawling(reviews)

    def run(self) -> None:
        """전체 리뷰 크롤링을 실행하는 함수"""
        logger.debug(f"==========START CRAWLING==========")
        logger.debug(f"URL: {self.url}")

        self.driver = self._open_chrome()
        reviews = self._sort_and_scroll()

        logger.debug(f"{len(reviews)}개의 리뷰를 추출합니다. >>> 텍스트인 리뷰만 수집됩니다.")

        self._crawling(reviews)
        logger.debug(f"수집된 리뷰는 {len(self.json_data)}개 입니다.")
        
        n_add = self.n_target - len(self.json_data)
        if self.no_data:
            logger.debug(f"더 이상 수집할 데이터가 없습니다.")
        elif n_add > 0:
            n_click = int(n_add // 10) + 1
            logger.debug(f"목표 수집량을 위해 추가 수집합니다.")
            self._add_crawling(n_click)
        
        self.data = pd.json_normalize(self.json_data)
        logger.debug(f"최종 수집된 리뷰는 {len(self.json_data)}개 입니다.")
        logger.debug(f"========== END  CRAWLING==========")

    def save_data(self, save_path: str) -> None:
        """리뷰 데이터를 save_path에 저장하는 메서드

        Args:
            save_path (str): 저장할 파일 경로
        """
        if isinstance(save_path, WindowsPath):
            save_path = save_path.as_posix()

        if save_path[-4:] == "json":
            with open(save_path, "w", encoding="utf-8") as json_file:
                json.dump(self.json_data, json_file, indent=4, ensure_ascii=False)
        else:
            self.data.to_csv(save_path, index=False, encoding="utf-8")

        logger.debug(f"SUCCESS! PATH: {save_path}")

### 실행

In [17]:
url = "https://pcmap.place.naver.com/restaurant/1804344332/review/visitor" # 리뷰 적은 곳
# url = "https://pcmap.place.naver.com/restaurant/13166754/review/visitor" # 리뷰 많은 곳

crawler = ReviewCrawler(url, n_target=200)
crawler.run()

DEBUG: ==========START CRAWLING==========
DEBUG: URL: https://pcmap.place.naver.com/restaurant/1804344332/review/visitor
DEBUG: 15개의 리뷰를 추출합니다. >>> 텍스트인 리뷰만 수집됩니다.
DEBUG: 수집된 리뷰는 13개 입니다.
DEBUG: 더 이상 수집할 데이터가 없습니다.
DEBUG: 최종 수집된 리뷰는 13개 입니다.
DEBUG: ========== END  CRAWLING==========


In [3]:
crawler.save_data("data/naver_review_raw_data.json")
crawler.save_data("data/naver_review_raw_data.csv")

DEBUG: SUCCESS! PATH=data/naver_review_raw_data.json
DEBUG: SUCCESS! PATH=data/naver_review_raw_data.csv


### 결과

In [18]:
crawler.json_data[:2]

[{'reviewer': '쏭쏭쏭4725',
  'review': '케이크 너무 이쁘고 만족이에요\U0001fa77\U0001fa77 요새 주문제작 케이크 다 가격대가 있긴하지만 좀 고민했는데 디자인이 여기만큼 확 끌리는데가 없어서 했는데... 진짜 완전 대만족입니다!! 레터링 케이크 주문제작 케이크 비싼 돈주고 해서 맛있게 먹은적 단 한번도 없는데.. 그냥 사진찍기용이였지 진\n짜 맛까지 만족한적 한번도 없거든요.. 다 버렸는데 요기는 진짜 케이크도 존맛탱입니다\U0001fa77 여기를 왜 이제서야 안건지 ㅠㅠㅠㅠ 다음에도 꼭 여기서 할게요\U0001fa77',
  'additional_info': '예약 후 이용, 대기 시간 바로 입장, 기념일, 부모님',
  'date': '2024년 4월 26일 금요일',
  'n_visit': '1번째 방문',
  'auth_method': '영수증'},
 {'reviewer': 'mini131',
  'review': '친절하게 잘 해주셨어용',
  'additional_info': '예약 없이 이용, 대기 시간 바로 입장, 친목, 혼자',
  'date': '2023년 12월 10일 일요일',
  'n_visit': '1번째 방문',
  'auth_method': '영수증'}]

In [19]:
print(crawler.data.shape)
crawler.data.head()

(13, 6)


,reviewer,review,additional_info,date,n_visit,auth_method
0,쏭쏭쏭4725,케이크 너무 이쁘고 만족이에요🩷🩷 요새 주문제작 케이크 다 가격대가 있긴하지만 좀 ...,"예약 후 이용, 대기 시간 바로 입장, 기념일, 부모님",2024년 4월 26일 금요일,1번째 방문,영수증
1,mini131,친절하게 잘 해주셨어용,"예약 없이 이용, 대기 시간 바로 입장, 친목, 혼자",2023년 12월 10일 일요일,1번째 방문,영수증
2,메주콩16,생신케이크로 생화케이크 주문했는데 진짜 이쁘고 엄마가 너무너무 좋아하셨습니다 하루전...,,2023년 4월 27일 목요일,1번째 방문,결제내역
3,나0829,맛있어요,,2023년 2월 23일 목요일,1번째 방문,영수증
4,ony****,"안녕하세요, 사장님! 리뷰를 남긴다는게 깜빡해서 지금에야 남기네요 ㅠㅠ 급하게 케이...",,2022년 12월 18일 일요일,1번째 방문,영수증
